In [ ]:
!pip install -q pythainlp

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from pythainlp.util import normalize
from pythainlp import word_tokenize

# **Import Text**

In [ ]:
#Gdown
!gdown --id 148vJ_8Tjoh-HZAuHBg_a5x0ebveoL_Mf

Downloading...
From: https://drive.google.com/uc?id=148vJ_8Tjoh-HZAuHBg_a5x0ebveoL_Mf
To: /content/AIFORTHAI-LST20Corpus.tar.gz
100% 13.6M/13.6M [00:00<00:00, 46.4MB/s]


In [ ]:
!tar -xvf /content/AIFORTHAI-LST20Corpus.tar.gz

Streaming output truncated to the last 5000 lines.
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/train/

In [ ]:
#Directory for each from gdrive 
LST20_PATH_TRAIN = glob.glob('/content/LST20_Corpus/train/*')
LST20_PATH_EVAL = glob.glob('/content/LST20_Corpus/eval/*')
LST20_PATH_TEST = glob.glob('/content/LST20_Corpus/test/*')

In [ ]:
line_list = []
with open('/content/LST20_Corpus/train/T00128.txt', 'r') as f:
    for line in f:
        if line == '\n':
            continue
        line = line.strip().split('\t')
        line_list.append(line)

In [ ]:
def Dataframe_from_LST20_Path(list_path):
    """return text file in directory to datframe
    input: folder_path which contains txt file
    output: dataframe"""
    line_list = []    
    for item in list_path:     
      # if item[0] != '.': # if have a '.' file in directory
        with open(item,'r') as f:
            for line in f:
                if line == '\n': #fill newline
                    continue                    
                line = line.strip().split('\t')          
                line_list.append(line)
                    
    df = pd.DataFrame(line_list,columns=['word','POS','name_entity-type','sentencs_seg']) #4 columns refer to 3 task of nlp
    #in case column NAN because newline, will be filled by some word
    df['POS'] = df['POS'].fillna("newline")
    df['name_entity-type'] = df['name_entity-type'].fillna("newline")
    df['sentencs_seg'] = df['sentencs_seg'].fillna("newline")
    return df

In [ ]:
eval_df = Dataframe_from_LST20_Path(LST20_PATH_EVAL)
train_df = Dataframe_from_LST20_Path(LST20_PATH_TRAIN)
test_df = Dataframe_from_LST20_Path(LST20_PATH_TEST)

In [ ]:
train_df

,word,POS,name_entity-type,sentencs_seg
0,"""",PU,O,B_CLS
1,มิ่งขวัญ,NN,B_PER,I_CLS
2,"""",PU,O,I_CLS
3,พร้อม,PS,O,I_CLS
4,บอร์ด,NN,O,I_CLS
...,...,...,...,...
2714721,ไว้,AX,O,I_CLS
2714722,กับ,PS,O,I_CLS
2714723,พนักงาน,NN,O,I_CLS
2714724,สอบสวน,VV,O,I_CLS


In [ ]:
def char_split_word_tag(df):
    list_word = df['word'].tolist()
    word_tag = []
    for word in list_word:
        if word == ' ':
            word_tag.append(('', 'O'))
            continue
        for c in word:
            if c == '"' or c == '':
                word_tag.append((c, 'I_WORD'))
                continue
            elif c == ' ':
                word_tag.append(('', 'O'))
            elif c == word[0]:
                word_tag.append((c, 'B_WORD'))
            elif c == word[-1]:
                word_tag.append((c, 'E_WORD'))
            else:
                word_tag.append((c, 'I_WORD'))
    return pd.DataFrame(word_tag, columns=['Id', 'Predicted'])

In [ ]:
char_df = char_split_word_tag(train_df)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2714726 entries, 0 to 2714725
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   word              object
 1   POS               object
 2   name_entity-type  object
 3   sentencs_seg      object
dtypes: object(4)
memory usage: 82.8+ MB


In [ ]:
#export to save file in csv format
eval_df.to_csv('eval_org_df_lst20.csv',index=False)
train_df.to_csv('train_org_df_lst20.csv',index=False)
test_df.to_csv('test_org_df_lst20.csv',index=False)

## Using Attacut on submission

In [ ]:
!pip install attacut -qq
!pip install deepcut -qq

In [ ]:
f = open('/content/ws_test.txt')
text = f.read()
f.close()

In [ ]:
text

'ที่ยังสถานการณ์ยังไม่คลี่คลายอาจส่งผลกระทบการค้าชายแดนไทย - กัมพูชา ว่า เท่าที่เข้าไปดำเนินการตรวจสอบยังไม่พบว่ามีการปิดด่านบริเวณดังกล่าว และที่สำคัญการค้าระหว่างไทยและกัมพูชาส่วนใหญ่จะมีปริมาณมากที่ด่านปอยเปต อ. อรัญประเทศ จ. สระแก้ว มากกว่าด่านอื่น ๆ ส่วนที่กันทรลักษ์เป็นด่านขนาดเล็กจะเน้นด้านท่องเที่ยว จึงไม่น่าส่งผลกระทบต่อบรรยากาศการค้าชายแดนไทย - กัมพูชาแต่อย่างใด อย่างไรก็ตาม แม้จะมีปัญหาความขัดแย้งเรื่องเขาพระวิหาร แต่ปริมาณการค้าชายแดนไทย - กัมพูชาในปีที่ผ่านมายังคงเพิ่มขึ้น โดยมีจำปริมาณทั้งสิ้น 48,406 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับปี 2549 ร้อยละ 0.2 โดยเป็นการส่งออกจากไทยทั้งสิ้น 46,718 ล้านบาท ขณะที่มีการนำเข้าสินค้าจากกัมพูชาเพียง 1,689 ล้านบาทเท่านั้น ทั้งนี้ ในช่วง 5 เดือนแรกปีนี้ ( ม.ค. - พ.ค. 2551 ) การค้าทั้งสองประเทศมีมูลค่ารวมทั้งสิ้น 29,677 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับช่วงเดียวกันของปีที่ผ่านมาร้อยละ 59.2 โดยไทยส่งออกมูลค่าทั้งสิ้น 28,680 ล้านบาท และมีการนำเข้าเพียง 997 ล้านบาท ถกนอกรอบ " ไทย - เขมร " 8 ชม. สรุป " ตรึงกำลังทหาร - ห้ามยิง " ผลการประชุมร่วมไทย - 

In [ ]:
count = 0
word_list = []
attacut_token = word_tokenize(normalize(text), engine="attacut", keep_whitespace=True)

print(attacut_token)

['ที่', 'ยัง', 'สถานการณ์', 'ยัง', 'ไม่', 'คลี่คลาย', 'อาจ', 'ส่ง', 'ผล', 'กระทบ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', ' ', 'ว่า', ' ', 'เท่า', 'ที่', 'เข้า', 'ไป', 'ดำเนิน', 'การ', 'ตรวจสอบ', 'ยัง', 'ไม่', 'พบ', 'ว่า', 'มี', 'การ', 'ปิด', 'ด่าน', 'บริเวณ', 'ดัง', 'กล่าว', ' ', 'และ', 'ที่', 'สำคัญ', 'การ', 'ค้า', 'ระหว่าง', 'ไทย', 'และ', 'กัมพูชา', 'ส่วน', 'ใหญ่', 'จะ', 'มี', 'ปริมาณ', 'มาก', 'ที่', 'ด่านปอยเปต', ' ', 'อ. อรัญประเทศ', ' ', 'จ. สระแก้ว', ' ', 'มาก', 'กว่า', 'ด่าน', 'อื่น', ' ', 'ๆ', ' ', 'ส่วน', 'ที่', 'กัน', 'ทรลักษ์', 'เป็น', 'ด่าน', 'ขนาด', 'เล็ก', 'จะ', 'เน้น', 'ด้าน', 'ท่องเที่ยว', ' ', 'จึง', 'ไม่', 'น่า', 'ส่ง', 'ผล', 'กระทบ', 'ต่อ', 'บรรยากาศ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', 'แต่อย่างใด', ' ', 'อย่าง', 'ไร', 'ก็ตาม', ' ', 'แม้', 'จะ', 'มี', 'ปัญหา', 'ความ', 'ขัดแย้ง', 'เรื่อง', 'เขาพระวิหาร', ' ', 'แต่', 'ปริมาณ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', 'ใน', 'ปี', 'ที่', 'ผ่าน', 'มา', 'ยังคง', 'เพ

In [ ]:
test_df = pd.DataFrame(attacut_token, columns=['word'])

In [ ]:
sub = char_split_word_tag(test_df)
sub

,Id,Predicted
0,ท,B_WORD
1,ี,I_WORD
2,่,E_WORD
3,ย,B_WORD
4,ั,I_WORD
...,...,...
37243,ย,E_WORD
37244,แ,B_WORD
37245,ล,I_WORD
37246,้,I_WORD


In [ ]:
sub['Id'] = sub.index + 1

In [ ]:
sub

,Id,Predicted
0,1,B_WORD
1,2,I_WORD
2,3,E_WORD
3,4,B_WORD
4,5,I_WORD
...,...,...
37243,37244,E_WORD
37244,37245,B_WORD
37245,37246,I_WORD
37246,37247,I_WORD


In [ ]:
ready_to_submit = sub[sub['Predicted'] != 'O']

In [ ]:
ready_to_submit

,Id,Predicted
0,1,B_WORD
1,2,I_WORD
2,3,E_WORD
3,4,B_WORD
4,5,I_WORD
...,...,...
37243,37244,E_WORD
37244,37245,B_WORD
37245,37246,I_WORD
37246,37247,I_WORD


In [ ]:
# ready_to_submit.to_csv('submit_deepcut_full.csv', index=False)

In [ ]:
def char_split_text_to_word(text):

    word_tag = []
    text_split = text.split(' ')
    print(text_split)
    # isfirsttime = True
    for word in text_split:
        # Fix bug begining word
        isbegining = True
        # if not isfirsttime:
        #     word_tag.append(('', 'O'))

        # isfirsttime = False
        for c in word:
            if c == '"' or c == '':
                word_tag.append((c, 'I_WORD'))
                continue

            elif c == ' ':
                word_tag.append(('', 'O'))
            elif c == word[0] and isbegining:
                word_tag.append((c, 'B_WORD'))
                isbegining = False
            elif c == word[-1]:
                word_tag.append((c, 'E_WORD'))
            else:
                word_tag.append((c, 'I_WORD'))
            
    return pd.DataFrame(word_tag, columns=['Id', 'Predicted'])

In [ ]:
def ground_truth_from_text_file(path_txt_file):

    f = open(path_txt_file, 'r')
    data_row = list()

    for word in f:
        if word == '\n':
            continue
        data_row.append(word.strip().split('\t'))

    df_ =  pd.DataFrame(data_row, columns=['word','POS','name_entity-type','sentencs_seg'])
    word_tag = list()
    list_of_word = df_['word'].tolist()

    for word in list_of_word:
        isfirsttime = True
        index_last_char = word.rindex(word[-1])

        for idx, c in enumerate(word):
            if c == '_':
                word_tag.append((c, 'O'))
            elif isfirsttime:
                word_tag.append((c, 'B_WORD'))
            elif idx == index_last_char:
                word_tag.append((c, 'E_WORD'))
            else:
                word_tag.append((c, 'I_WORD'))
            isfirsttime = False
    f.close()
    return pd.DataFrame(word_tag, columns=['Id', 'Predicted'])


def inference_text(text, engine='attacut'):

    if engine == 'attacut':
        token_list = word_tokenize(normalize(text), engine="attacut", keep_whitespace=True)
    if engine == 'deepcut':
        token_list = word_tokenize(normalize(text), engine="deepcut", keep_whitespace=True)
    if engine == 'newmm':
        token_list = word_tokenize(normalize(text), engine="newmm", keep_whitespace=True)

    word_tag = []
    isfirsttime = True
    for word in token_list:
        isbegining = True
        if not isfirsttime:
            if word == ' ':
                word_tag.append(('', 'O'))
                continue

        isfirsttime = False
        for c in word:
            if c == '"' or c == '':
                word_tag.append((c, 'I_WORD'))
                continue

            elif c == ' ':
                word_tag.append(('', 'O'))
            elif c == word[0] and isbegining:
                word_tag.append((c, 'B_WORD'))
                isbegining = False
            elif c == word[-1]:
                word_tag.append((c, 'E_WORD'))
            else:
                word_tag.append((c, 'I_WORD'))
            
    return pd.DataFrame(word_tag, columns=['Id', 'Predicted'])

In [ ]:
def text_file_to_paragraph(file_path):
    line_list = []
    # file_path = '/content/LST20_Corpus/train/T00126.txt'
    with open(file_path, 'r') as f:
        for line in f:
            if line == '\n': #fill newline
                continue                    
            line = line.strip().split('\t')          
            line_list.append(line)
    return pd.DataFrame(line_list, columns=['word','POS','name_entity-type','sentencs_seg'])
    df = pd.DataFrame(line_list, columns=['word','POS','name_entity-type','sentencs_seg'])
    list_of_word = df['word'].tolist()



In [ ]:
temp_df = text_file_to_paragraph('/content/LST20_Corpus/train/T00126.txt')

In [ ]:
['' for w in temp_df['word'].tolist()]

['สุรยุทธ์',
 'ยัน',
 'ปฏิเสธ',
 'ลงนาม',
 '_',
 'MOU',
 '_',
 'กับ',
 'อียู',
 'ไม่',
 'กระทบ',
 'สัมพันธ์',
 '1',
 '_',
 'กันยายน',
 '_',
 '2550',
 '_',
 '12:21',
 '_',
 'น.',
 'นายก',
 'ฯ',
 '_',
 'ย้ำ',
 'ไม่',
 'ลงนาม',
 'ใน',
 '_',
 'MOU',
 '_',
 'ร่วมกับ',
 'อียู',
 'กรณีที่',
 'จะ',
 'เข้า',
 'มา',
 'สังเกตการณ์',
 'การ',
 'เลือกตั้ง',
 'ของ',
 'ไทย',
 'ใน',
 'ลักษณะ',
 'เข้า',
 'มา',
 'ตรวจสอบ',
 'ยัน',
 'ไทย',
 'ไม่',
 'ได้',
 'ล้มเหลว',
 'ถึง',
 'ขั้น',
 'นั้น',
 'เชื่อ',
 'การ',
 'ปฏิเสธ',
 'เรื่อง',
 'ดังกล่าว',
 'ไม่',
 'กระทบ',
 'สัมพันธ์',
 'ที่',
 'มี',
 'ต่อ',
 'กัน',
 'พร้อม',
 'หนุน',
 'กกต.',
 'จัด',
 'เลือกตั้ง',
 'ให้',
 'บริสุทธิ์',
 'ยุติธรรม',
 '(',
 '1',
 '_',
 'ก.ย.',
 ')',
 '_',
 'พล.อ.',
 'สุรยุทธ์',
 '_',
 'จุลานนท์',
 '_',
 'กล่าว',
 'รายการ',
 'เปิด',
 'บ้านพิษณุโลก',
 'ออกอากาศ',
 'ทาง',
 'สถานี',
 'วิทยุ',
 'โทรทัศน์',
 'แห่ง',
 'ประเทศ',
 'ไทย',
 'ช่อง',
 '_',
 '11',
 '_',
 'กรม',
 'ประชาสัมพันธ์',
 'กรณีที่',
 'สหภาพ',
 'ยุโรป',
 '_',
 '(',
 'อียู',

In [ ]:
text

'ที่ยังสถานการณ์ยังไม่คลี่คลายอาจส่งผลกระทบการค้าชายแดนไทย - กัมพูชา ว่า เท่าที่เข้าไปดำเนินการตรวจสอบยังไม่พบว่ามีการปิดด่านบริเวณดังกล่าว และที่สำคัญการค้าระหว่างไทยและกัมพูชาส่วนใหญ่จะมีปริมาณมากที่ด่านปอยเปต อ. อรัญประเทศ จ. สระแก้ว มากกว่าด่านอื่น ๆ ส่วนที่กันทรลักษ์เป็นด่านขนาดเล็กจะเน้นด้านท่องเที่ยว จึงไม่น่าส่งผลกระทบต่อบรรยากาศการค้าชายแดนไทย - กัมพูชาแต่อย่างใด อย่างไรก็ตาม แม้จะมีปัญหาความขัดแย้งเรื่องเขาพระวิหาร แต่ปริมาณการค้าชายแดนไทย - กัมพูชาในปีที่ผ่านมายังคงเพิ่มขึ้น โดยมีจำปริมาณทั้งสิ้น 48,406 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับปี 2549 ร้อยละ 0.2 โดยเป็นการส่งออกจากไทยทั้งสิ้น 46,718 ล้านบาท ขณะที่มีการนำเข้าสินค้าจากกัมพูชาเพียง 1,689 ล้านบาทเท่านั้น ทั้งนี้ ในช่วง 5 เดือนแรกปีนี้ ( ม.ค. - พ.ค. 2551 ) การค้าทั้งสองประเทศมีมูลค่ารวมทั้งสิ้น 29,677 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับช่วงเดียวกันของปีที่ผ่านมาร้อยละ 59.2 โดยไทยส่งออกมูลค่าทั้งสิ้น 28,680 ล้านบาท และมีการนำเข้าเพียง 997 ล้านบาท ถกนอกรอบ " ไทย - เขมร " 8 ชม. สรุป " ตรึงกำลังทหาร - ห้ามยิง " ผลการประชุมร่วมไทย - 

In [ ]:
with open('/content/LST20_Corpus/train/T00126.txt') as f:
    for line in f:
        print(line.word())

สุรยุทธ์	NN	B_PER	B_CLS

ยัน	VV	O	I_CLS

ปฏิเสธ	VV	O	I_CLS

ลงนาม	VV	O	I_CLS

_	PU	O	I_CLS

MOU	NN	O	I_CLS

_	PU	O	I_CLS

กับ	PS	O	I_CLS

อียู	NN	B_ORG	I_CLS

ไม่	NG	O	I_CLS

กระทบ	VV	O	I_CLS

สัมพันธ์	NN	O	E_CLS



1	NU	B_DTM	B_CLS

_	PU	I_DTM	I_CLS

กันยายน	NN	I_DTM	I_CLS

_	PU	I_DTM	I_CLS

2550	NU	E_DTM	I_CLS

_	PU	O	I_CLS

12:21	NU	B_DTM	I_CLS

_	PU	I_DTM	I_CLS

น.	CL	E_DTM	E_CLS



นายก	NN	O	B_CLS

ฯ	PU	O	I_CLS

_	PU	O	I_CLS

ย้ำ	VV	O	I_CLS

ไม่	NG	O	I_CLS

ลงนาม	VV	O	I_CLS

ใน	PS	O	I_CLS

_	PU	O	I_CLS

MOU	NN	O	I_CLS

_	PU	O	I_CLS

ร่วมกับ	PS	O	I_CLS

อียู	NN	B_ORG	E_CLS

กรณีที่	CC	O	B_CLS

จะ	AX	O	I_CLS

เข้า	AV	O	I_CLS

มา	AV	O	I_CLS

สังเกตการณ์	VV	O	I_CLS

การ	FX	O	I_CLS

เลือกตั้ง	VV	O	I_CLS

ของ	PS	O	I_CLS

ไทย	NN	B_LOC	I_CLS

ใน	PS	O	I_CLS

ลักษณะ	NN	O	I_CLS

เข้า	AV	O	I_CLS

มา	AV	O	I_CLS

ตรวจสอบ	VV	O	E_CLS



ยัน	VV	O	B_CLS

ไทย	NN	B_LOC	I_CLS

ไม่	NG	O	I_CLS

ได้	AX	O	I_CLS

ล้มเหลว	VV	O	I_CLS

ถึง	VV	O	I_CLS

ขั้น	NN	O	I_CLS

นั้น	AJ	O	I_CLS

เชื่อ	VV	O	I_CLS

การ	FX

In [ ]:
inference_df = ground_truth_from_text_file('/content/LST20_Corpus/train/T00126.txt')

pred_df = inference_text(text, engine='attacut')

In [ ]:
inference_df

,Id,Predicted
0,ส,B_WORD
1,ุ,I_WORD
2,ร,I_WORD
3,ย,I_WORD
4,ุ,I_WORD
...,...,...
1658,เ,B_WORD
1659,ม,I_WORD
1660,ื,I_WORD
1661,อ,I_WORD


# Accuracy F1-Score

In [ ]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
inference_df[inference_df['Predicted'] == 'B_WORD'].index

Int64Index([   0,    8,   11,   17,   23,   27,   30,   34,   37,   42,
            ...
            1591, 1612, 1619, 1623, 1626, 1634, 1639, 1643, 1651, 1652],
           dtype='int64', length=377)

In [ ]:
inference_df

,Id,Predicted
0,ส,B_WORD
1,ุ,I_WORD
2,ร,I_WORD
3,ย,I_WORD
4,ุ,I_WORD
...,...,...
1658,เ,I_WORD
1659,ม,I_WORD
1660,ื,I_WORD
1661,อ,I_WORD


In [ ]:
def f1_score_(groud_truth_df, inference_df):

    pred_B_index = set(inference_df[inference_df['Predicted'] == 'B_WORD'].index)
    true_B_index = set(groud_truth_df[groud_truth_df['Predicted'] == 'B_WORD'].index)
    TP = len(pred_B_index.intersection(true_B_index))

    FN = len(pred_B_index.difference(true_B_index))
    FP = len(true_B_index.difference(pred_B_index))

    return TP / (TP + 0.5*(FP + FN))

In [ ]:
acc = f1_score_(groud_truth_df, inference_df)
acc

0.9299867899603699

In [ ]:
text_T00126 = ''.join(df_T00126['word'].tolist()).replace('_', ' ')
text_T00126

'สุรยุทธ์ยันปฏิเสธลงนาม MOU กับอียูไม่กระทบสัมพันธ์1 กันยายน 2550 12:21 น.นายกฯ ย้ำไม่ลงนามใน MOU ร่วมกับอียูกรณีที่จะเข้ามาสังเกตการณ์การเลือกตั้งของไทยในลักษณะเข้ามาตรวจสอบยันไทยไม่ได้ล้มเหลวถึงขั้นนั้นเชื่อการปฏิเสธเรื่องดังกล่าวไม่กระทบสัมพันธ์ที่มีต่อกันพร้อมหนุนกกต.จัดเลือกตั้งให้บริสุทธิ์ยุติธรรม(1 ก.ย.) พล.อ.สุรยุทธ์ จุลานนท์ กล่าวรายการเปิดบ้านพิษณุโลกออกอากาศทางสถานีวิทยุโทรทัศน์แห่งประเทศไทยช่อง 11 กรมประชาสัมพันธ์กรณีที่สหภาพยุโรป (อียู) จะขอเข้ามาสังเกตการณ์การเลือกตั้งของไทยโดยจะขอลงนามในบันทึกความเข้าใจ(MOU) ร่วมกันว่าเป็นเรื่องที่คณะกรรมการการเลือกตั้งซึ่งนายอภิชาต สุขัคคานนท์ประธานกกต.ได้บอกว่าเรื่องของความตกลงร่วมมือกันมี 2 ลักษณะคือหากเป็นลักษณะทั่วไปแล้วจะไม่เข้ามายุ่งเกี่ยวกับเรื่องของภายในคล้ายๆ ผู้สังเกตการณ์ส่วนอีกลักษณะหนึ่งที่ทางอียูเสนอมานั้นเป็นลักษณะที่ขอเข้ามาตรวจสอบเขาไม่ได้ให้เหตุผลนะครับซึ่งเราคงไม่สามารถจะยอมรับได้เขามายุ่งเกี่ยวกับเรื่องภายในของเรามากจนเกินไปเราไม่ได้เป็นประเทศที่ถือว่ามีความล้มเหลวในด้านการบริหารทุกสิ่งทุกอย่างของเรานี้ยังดำเนินการไป

In [ ]:
y_pred = inference_df['Predicted'].tolist()
y_true = groud_truth_df['Predicted'].tolist()

y_pred_oh = pd.get_dummies(y_true).values
y_true_oh = pd.get_dummies(y_pred).values

f1_score(y_true_oh, y_pred_oh, average='macro')


0.944970022093296

In [ ]:
f = open('/content/LST20_Corpus/train/T00126.txt', 'r')
data_row = list()

for word in f:
    if word == '\n':
        continue
    data_row.append(word.strip().split('\t'))
data_row